In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Does nn.Conv2d init work well?

In [0]:
#export
from exp.nb_02 import *

def get_data():
    path = Path('data/mnist.pkl.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): 
    return (x-m)/s

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [0]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
# resizing vector to 2d tensor to fit into convalution layer
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [7]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [0]:
l1 = nn.Conv2d(1, nh, 5) # 1 chanel input, 32 channel output with 5x5 kernel size

In [0]:
# small sammple for testing purposes
x = x_valid[:100]

In [10]:
x.shape

torch.Size([100, 1, 28, 28])

In [0]:
def stats(x): 
    return x.mean(),x.std()

In [12]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [13]:
stats(l1.weight),stats(l1.bias)

((tensor(-0.0059, grad_fn=<MeanBackward0>),
  tensor(0.1161, grad_fn=<StdBackward0>)),
 (tensor(-0.0220, grad_fn=<MeanBackward0>),
  tensor(0.1206, grad_fn=<StdBackward0>)))

In [0]:
t = l1(x)

In [15]:
stats(t)

(tensor(-0.0376, grad_fn=<MeanBackward0>),
 tensor(0.6506, grad_fn=<StdBackward0>))

standard deviation is not close to 1, so we compare the kaiming uniform with kaiming normal. 

**Kaiming Normal uses Leaky ReLu layer**

In [19]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(0.0019, grad_fn=<MeanBackward0>),
 tensor(1.0800, grad_fn=<StdBackward0>))

In [0]:
import torch.nn.functional as F

In [0]:
def f1(x,a=0): 
    return F.leaky_relu(l1(x),a)

In [22]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5173, grad_fn=<MeanBackward0>),
 tensor(1.0073, grad_fn=<StdBackward0>))

In [23]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))
# after relu it's even worse

(tensor(0.2396, grad_fn=<MeanBackward0>),
 tensor(0.4403, grad_fn=<StdBackward0>))

In [24]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

**Implementing our own Kaiming initializer**

In [25]:
# receptive field size; take one of the lement and call numel() will give the 5x5 num of elements.
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [26]:
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

In [27]:
fan_in  = ni*rec_fs # input channels * num of receptors
fan_out = nf*rec_fs # output channels * num of receptors
fan_in,fan_out

(25, 800)

In [0]:
# kaiming init formula
def gain(a): 
    return math.sqrt(2.0 / (1 + a**2))

In [29]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [30]:
# we have implemented kaiming normal but cnv2d uses kaiming uniform; chacking the standard deviation for random numbers 
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5779)

In [31]:
# equal to 1/√3; which we use to make normal distribution to uniform distribution
1/math.sqrt(3.)

0.5773502691896258

In [0]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [33]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5211, grad_fn=<MeanBackward0>),
 tensor(0.9394, grad_fn=<StdBackward0>))

In [34]:
kaiming2(l1.weight, a=math.sqrt(5.)) # √5 used in pytorch
stats(f1(x))

(tensor(0.2520, grad_fn=<MeanBackward0>),
 tensor(0.4290, grad_fn=<StdBackward0>))

In [0]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [0]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [0]:
# sample dependent variables to test
y = y_valid[:100].float()

In [38]:
t = m(x)
stats(t)

(tensor(-0.0451, grad_fn=<MeanBackward0>),
 tensor(0.0102, grad_fn=<StdBackward0>))

In [0]:
l = mse(t,y)
l.backward()

In [40]:
stats(m[0].weight.grad)

(tensor(0.0262), tensor(0.0296))

In [0]:
init.kaiming_uniform_??

In [0]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [42]:
t = m(x)
stats(t)

(tensor(0.3679, grad_fn=<MeanBackward0>),
 tensor(0.2916, grad_fn=<StdBackward0>))

In [43]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.0645), tensor(0.4492))

## Export

In [44]:
!chmod 755 ./notebook2script.py
!./notebook2script.py 02a_why_sqrt5.ipynb

Converted 02a_why_sqrt5.ipynb to exp/nb_02a.py
